In [9]:
import requests
import pandas as pd
import tqdm, sys, os
import numpy as np
from sqlalchemy import create_engine
import yfinance as yf

!pip install PyMySQL

In [10]:
sql_engine = create_engine('mysql+pymysql://root:@localhost/tp_curso_2')
sql_conn = sql_engine.connect()

In [11]:
sp500_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_wiki = sp500_wiki.loc[~sp500_wiki.Symbol.isin(['BRK.B' ,'BF.B'])]
sp500_wiki

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [12]:
sp500_wiki.to_sql(con=sql_conn, name='sp500_tickers', if_exists='replace')

In [13]:
tickers = pd.read_sql('SELECT Symbol FROM sp500_tickers', sql_conn)

In [14]:
sp500_tickers = tickers.Symbol.tolist()

In [15]:
def getDataM(listado, start='2000-01-01', interval='1d', end=None):
    data = yf.download(listado, start=start, end=end, interval=interval, auto_adjust=True)
    return data.swaplevel(i=1, j=0, axis=1)

In [16]:
df_sp500 = getDataM(sp500_tickers, start='2018-01-01', interval='1d')

[*********************100%***********************]  503 of 503 completed


In [18]:
tablas = []
with tqdm.tqdm(total=len(sp500_tickers), file=sys.stdout) as pbar:
    for ticker in sp500_tickers:
        pbar.update()
        tabla = df_sp500[ticker].copy()
        tabla['symbol'] = ticker
        tabla['sma_21'] = tabla.Close.rolling(21).mean()
        tabla['Distancia'] = (tabla.Close - tabla.sma_21) / tabla.Close * 100
        tabla = tabla.dropna().round(2)
        tablas.append(tabla)
        
        dif = tabla['Close'].diff()
        win = pd.DataFrame(np.where(dif > 0, dif, 0))
        loss = pd.DataFrame(np.where(dif < 0, abs(dif), 0))
        ema_win = win.ewm(alpha=1 / 14).mean()
        ema_loss = loss.ewm(alpha=1 / 14).mean()
        rs = ema_win / ema_loss
        rsi = 100 - (100 / (1 + rs))
        rsi.index = tabla.index
        tabla['rsi'] = rsi

diario_m21 = pd.concat(tablas)        
diario_m21

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:13<00:00, 38.13it/s]


,Close,High,Low,Open,Volume,symbol,sma_21,Distancia,rsi,Adj Close
Date,,,,,,,,,,
2018-01-31,229.57,231.98,228.74,230.47,3136400.0,MMM,225.04,1.97,NaN,NaN
2018-02-01,227.22,229.19,225.61,226.76,2665500.0,MMM,225.58,0.72,0.000000,NaN
2018-02-02,224.68,227.96,224.42,225.84,3925700.0,MMM,226.00,-0.58,0.000000,NaN
2018-02-05,212.10,223.98,207.12,223.15,6590500.0,MMM,225.68,-6.40,0.000000,NaN
2018-02-06,214.11,214.97,207.86,208.50,5108100.0,MMM,225.38,-5.26,11.315610,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-11-02,162.89,163.26,160.71,161.31,1516600.0,ZTS,162.20,0.42,52.674384,NaN
2020-11-03,163.14,165.49,162.90,164.64,1519500.0,ZTS,162.20,0.58,53.097454,NaN
2020-11-04,174.35,176.17,165.97,165.99,3403700.0,ZTS,162.82,6.61,67.239599,NaN


In [33]:
diario_m21.to_sql(con=sql_conn,  name='sp500_diario_ind', if_exists='replace')

In [20]:
q = 'ALTER TABLE sp500_diario_m21 ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
sql_conn.execute(q)

In [36]:
diariom21 = pd.read_sql('''SELECT symbol, Close, sma_21, Distancia, rsi
                FROM sp500_diario_ind 
                WHERE Date ="2020-11-05" AND Distancia > 0 AND Distancia < 3 AND rsi > 50 AND rsi < 60
                GROUP BY symbol'''
            , sql_conn)
diariom21

,symbol,Close,sma_21,Distancia,rsi
0,AAP,156.43,153.62,1.80,56.269108
1,AAPL,119.03,116.18,2.40,55.967411
2,ADBE,496.47,485.14,2.28,56.409537
3,AEE,82.47,82.40,0.09,53.268983
4,AFL,37.20,36.61,1.60,55.229588
...,...,...,...,...,...
118,WRB,62.90,62.76,0.22,51.747653
119,WRK,39.23,38.98,0.63,56.033968
120,XRAY,46.61,46.37,0.51,51.978295
121,XYL,90.82,89.72,1.22,56.808662


In [22]:
diariom21.to_sql(con=sql_conn, name='filtro_diario', if_exists='replace')

In [23]:
import datetime as dt
start = (dt.date.today() - dt.timedelta(days=30)).isoformat()
df_sp500_5m = getDataM(sp500_tickers, start=start, interval='5m')

[*********************100%***********************]  503 of 503 completed


In [24]:
tablas = []
with tqdm.tqdm(total=len(sp500_tickers), file=sys.stdout) as pbar:
    for ticker in sp500_tickers:
        pbar.update()
        tabla = df_sp500_5m[ticker].copy()
        tabla['symbol'] = ticker
        tabla['sma_50'] = tabla.Close.rolling(50).mean()
        tabla['Distancia'] = (tabla.Close - tabla.sma_50) / tabla.Close * 100
        tabla = tabla.dropna().round(2)
        tablas.append(tabla)
        
tabla_5m = pd.concat(tablas)
tabla_5m

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [00:07<00:00, 69.04it/s]


,Close,High,Low,Open,Volume,symbol,sma_50,Distancia
Datetime,,,,,,,,
2020-10-07 13:35:00-04:00,165.98,166.07,165.96,166.03,15414.0,MMM,166.22,-0.15
2020-10-07 13:40:00-04:00,166.13,166.16,165.79,166.00,15489.0,MMM,166.24,-0.07
2020-10-07 13:45:00-04:00,166.26,166.27,166.01,166.11,7730.0,MMM,166.25,0.00
2020-10-07 13:50:00-04:00,166.59,166.63,166.24,166.27,14491.0,MMM,166.27,0.19
2020-10-07 13:55:00-04:00,166.43,166.51,166.36,166.48,6958.0,MMM,166.28,0.09
...,...,...,...,...,...,...,...,...
2020-11-05 15:45:00-05:00,170.74,170.76,170.38,170.56,65765.0,ZTS,172.03,-0.76
2020-11-05 15:50:00-05:00,170.97,171.09,170.62,170.78,84408.0,ZTS,171.99,-0.60
2020-11-05 15:55:00-05:00,170.90,171.27,170.68,170.98,165671.0,ZTS,171.95,-0.61


In [25]:
tabla_5m.to_sql(con=sql_conn,  name='sp500_5minutos', if_exists='replace')

In [26]:
q = 'ALTER TABLE sp500_5minutos ADD id INT NOT NULL AUTO_INCREMENT PRIMARY KEY'
sql_conn.execute(q)

In [27]:
cinco = pd.read_sql('''SELECT symbol, Close, sma_50, Distancia
                FROM sp500_5minutos 
                WHERE Distancia > 0 and Datetime = "2020-11-05 15:55:00"
                GROUP BY symbol'''
            , sql_conn)
cinco

,symbol,Close,sma_50,Distancia
0,A,110.40,110.23,0.15
1,AAL,11.59,11.52,0.58
2,AAPL,118.98,118.44,0.45
3,ABBV,93.66,93.51,0.16
4,ADBE,496.47,494.40,0.42
...,...,...,...,...
160,XLNX,129.99,129.71,0.21
161,XRAY,46.61,46.31,0.65
162,XYL,90.82,90.80,0.02
163,YUM,100.75,100.27,0.48


In [28]:
cinco.to_sql(con=sql_conn, name='filtro_5', if_exists='replace')